### Essential helpers

In [1]:
import math
from pylab import *

set_printoptions(suppress=True)

def givens_rotation(v1, v2):
    """Calculate the Given rotation matrix"""
    # v1, v2 <type 'float'>
    
    if v1 == 0:
        cs, sn = 0, 1
    else:
        t = math.sqrt(v1*v1 + v2*v2)
        cs = abs(v1) / t
        sn = cs * v2 / v1
        
    return cs, sn  # <type 'float'>

def apply_givens_rotation(h, cs, sn, k):
    """Applying Givens Rotation to H col"""
    
    # apply for ith column
    for i in range(k-1):  # the first k-1
        temp = cs[i]*h[i] + sn[i]*h[i+1]
        h[i+1] = -sn[i]*h[i] + cs[i]*h[i+1]
        h[i] = temp
    
    # update the next sin cos values for rotation
    cs_k, sn_k = givens_rotation(h[k-1], h[k])  # in MATLAB, (h(k), h(k+1))
    
    # eliminate H[i,i-1] (H(i+1,i) in MATLAB)
    h[k-1] = cs_k*h[k-1] + sn_k*h[k]  # h(k) = cs_k*h(k) + sn_k*h(k+1);
    h[k] = 0.0                        # h(k+1) = 0.0;
    
    return h, cs_k, sn_k

### Arnoldi and GMRES

In [2]:
def arnoldi(A, Q, k):
    """Arnoldi Function"""
    
    # populate h (this part is missing in the original
    # MATLAB version because it is not necessary in MATLAB)
    h = zeros([k+1,1])
    
    q = A * Q[:,k-1]  # q = A * Q(:,k);
    for i in range(k):
        h[i] = transpose(q) * Q[:,i]
        q = q - float(h[i]) * Q[:,i]  # type conversion
    h[k] = linalg.norm(q, ord=2)  # h(k+1) = norm(q); in MATLAB
    q = q / h[k]                  # q = q / h(k+1); in MATLAB
    
    return h, q

def gmres(A, b, x, max_iterations=1000, threshold=1e-9):
    # A <type 'np.matrix'> - left-hand side of the linear system A(x)=b
    # b <type 'np.array'> - right-hand side of the linear system A(x)=b
    # x <type 'np.array'> - initial guess, given as a column vector in numpy
    # max_iterations <type 'int'> - criteria one for terminating the iteration
    # threshold <type 'float'> - criteria two for terminating the iteration
    
    n = size(A,0)
    m = max_iterations
    
    # some validity checks for safety
    assert n == len(b), 'Input sizes do not match!'
    assert n == len(x), 'Initial guess does not match the problem!'
    
    # use x as the initial vector
    r = b - A * x  # column vector
    b_norm = linalg.norm(b, ord=2)
    error = linalg.norm(r, ord=2) / b_norm
    
    # populate H and Q (this part is missing in the original
    # MATLAB code because it is not necessary in MATLAB)
    H = matrix(zeros([n+1,n]))
    Q = matrix(zeros([n,n+1]))
    
    # initialize the 1D vectors
    sn = zeros([m,1])  # in MATLAB, zeros(m,1)
    cs = zeros([m,1])  # the same for below
    e1 = zeros([n,1])
    e1[0] = 1
    e = [error]
    r_norm = linalg.norm(r, ord=2)  # float
    Q[:,0] = r / r_norm
    beta = r_norm * e1  # the same as el, n by 1
    
    for k in range(m):
        # run arnoldi
        # dummy_H, dummy_Q = arnoldi(A, Q, k+1)
        H[0:k+2,k], Q[:,k+1] = arnoldi(A, Q, k+1)
        # print dummy_H, dummy_Q to test the output from arnoldi(A, Q, k+1)
        # print H and Q to confirm they were assigned to the correct columns
        # note that these are two different processes
        # it is also important to examine the inputs, A and Q
        
        # eliminate the last element in H ith row
        # and update the rotation matrix
        H[0:k+2,k], cs[k], sn[k] = \
        apply_givens_rotation(H[0:k+2,k], cs, sn, k+1)
        # print H[0:k+2,k] to examine the output only
        # print H to confirm they were assigned to the correct columns
        
        # re-populate beta (this is another difference
        # from the original MATLAB code)
        if (k+1) >= len(beta):
            dummy = zeros([k+2,1])
            dummy[:len(beta)] = beta
            beta = dummy
        else:
            pass
        
        # update the residual vector
        beta[k+1] = -sn[k] * beta[k]
        beta[k] = cs[k] * beta[k]
        error = abs(beta[k+1]) / b_norm
        
        # save the error
        e.append(float(error))  # difference from the original MATLAB code
        
        if error <= threshold:
            break
        else:
            continue
    
    # calculate the result
    k += 1  # in the MATLAB version, you do not need to do anything with k
    # in Python, k has to be added by one so that k == n after the last iteration
    y,resid,rank,s = linalg.lstsq(H[0:k,0:k], beta[0:k])  # left division with numpy
    x = x + Q[:,0:k] * y
    
    # Types of output
    # x <type 'np.array'> - the final solution given as a column vector
    # e <type 'list'> - a list storing the convergence history
    return x, e